**PROGETTO DI MACHINE LEARNING**

TITOLO: REGRESSIONE POLINOMIALE CON L'USO DI K-MEANS

L'obiettivo del progetto è quello di sperimentare un metodo per ottimizzare il costo computazionale di un fit polinomiale, al costo di una perdita ridotta di precisione, sfruttando un prima fase in cui si utilizza k-means per ridurre i punti del dataset.

L'idea sfrutta delle ricerche relativamente superficiali (utilizzando chat-gpt e wikipedia) secondo le quali k-means ha una complessità attesa inferiore ristpetto ad un fit polinomiale se applicati ad un input di n punti con k prossimo al grado del fit polinomiale. Una volta deciso il grado, quindi prossimo a k, si procede ad effettuare k-means sugli n punti iniziali, ottenendo k cluster e, con tempo lineare sul prodotto (n*d), dove d è il numero di dimensioni dell'input, ad ottenere i k baricentri dei cluster; a questo punto si effettua una fit polinomiale su questi baricentri invece che sugli n punti iniziali. L'idea si basa su quella alla base del clustering: dato che ci si aspetta come evento ragionevole che la distanze tra i punti ed i baricentri dei rispettivi cluster di appartenenza siano relativamente ridotte, si suppone di avere una perdita di accuratezza del polynomial fit regionevolmente contenuta, a fronte, per alcuni input, di un importante risparmio in termini di complessità.

Nel seguito ci addentreremo meglio nei dettagli dell'idea in questione, cercando di verificarne(o smentirla) la validità e l'efficacia.

Autori: Federico Ventura e Francesco Petrangeli matricole 2025655 e 1991302

Iniziamo!

# *Importiamo le librerie:*

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn.metrics import r2_score, mean_squared_error
from numpy.polynomial.polynomial import Polynomial
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import silhouette_score
import pandas as pd
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

# Importiamo il primo datset

Nel corso del progetto utilizzeremo diversi dataset(di cui due costruiti da noi e due presi da scikit-learn), il più possibilmente reali e con caratteristiche diverse, per misurare al meglio la performance del modello su un problema vero con dati veri (e anche per prendere un bel voto:) )

In [ ]:
# Load the Data
data = np.loadtxt('population_over_time.dat')

Il dataset "**Poulation over time**" è un dataset costruito da noi che descrive l'evolversi di una popolazione lungo un certo periodo di tempo(in anni).

La spiegazione dettagliata del dataset(con relativo codice) è presente su un file che manderemo in allegato per email.

In [ ]:
# Splittiamo i dati in features e target
X = data[:, :-1]
y = data[:, -1]

In [ ]:
print(X.shape, y.shape)
print(X)
print(y)

Splittiamo adesso il nostro dataset in dati di training e data di test, dato che poi applicheremo k-means sul solo training set.

In [ ]:
# Splittiamo il Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# K-Means

Iniziamo adesso ad addentrarci nell'idea del progetto, applicando l'algoritmo K-means, che divide il dataset in cluster e per ogni cluster identifica un "**centroide**", ossia un punto medio.

Saranno proprio questi centroidi i nuovi punti su cui addestreremo il modello di regressione polinomiale.

In [ ]:
# Applichiamo K-Means
k = 15
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X_train)

# prendiamo i centroidi
centroids = kmeans.cluster_centers_

# Prediciamo i cluster labels per i dati di training
labels = kmeans.predict(X_train)

# prendiamo i valori target dei centroidi facendo la media dei valori target in ogni cluster
y_centroids = np.array([y_train[labels == i].mean() for i in range(k)])

# Sostituiamo il training set con i centroidi
X_train_centroids = centroids
y_train_centroids = y_centroids

# Calcoliamo e stampiamo il punteggio silhouette
silhouette_avg = silhouette_score(X_train, kmeans.labels_)
print(f'Punteggio silhouette per k={k}: {silhouette_avg}')

Dopo aver applicato k-means abbiamo adesso ottenuto un "**nuovo dataset**", ossia un dataset formato esclusivamente dai centroidi, su cui adesso applicheremo regressione polinomiale.

# Regressione polinomiale



Siamo finalmente arrivati al momento di applicare regressione polinomiale!

Applichiamo dunque adesso regressione polinomiale sul dataset formato dai centroidi. Facendo ciò stiamo risparmiando in termini di costo computazionale al costo di una piccola perdita di precisione.

In [ ]:
degree = 5
poly_model_centroids = Polynomial.fit(X_train_centroids.flatten(), y_train_centroids, degree)

# Fittiamo un modello polinomiale sul training set originale
poly_model_original = Polynomial.fit(X_train.flatten(), y_train, degree)

# valutiamo i modelli
y_pred_test_centroids = poly_model_centroids(X_test.flatten())
y_pred_test_original = poly_model_original(X_test.flatten())

# Calcoliamo R^2 sul test set per entrambi i modelli
r2_test_centroids = r2_score(y_test, y_pred_test_centroids)
r2_test_original = r2_score(y_test, y_pred_test_original)

# Calcoliamo l'MSE sul test set per entrambi
mse_test_centroids = mean_squared_error(y_test, y_pred_test_centroids)
mse_test_original = mean_squared_error(y_test, y_pred_test_original)

print(f'R^2 on test data (centroids): {r2_test_centroids}')
print(f'R^2 on test data (original): {r2_test_original}')
print(f'MSE on test data (centroids): {mse_test_centroids}')
print(f'MSE on test data (original): {mse_test_original}')

# Plottiamo i risultati
plt.figure(figsize=(14, 7))

# Generiamo un range di valori di x per plottare i polinomi
x_range = np.linspace(X_test.min(), X_test.max(), 500)

plt.subplot(1, 2, 1)
plt.scatter(X_test, y_test, color='blue', label='Actual Data')
plt.plot(x_range, poly_model_centroids(x_range.flatten()), color='red', label='Polynomial Fit (Centroids)', linewidth=3)
plt.xlabel('X')
plt.ylabel('y')
plt.legend()
plt.title('Polynomial Fit on Test Data (Centroids)')

plt.subplot(1, 2, 2)
plt.scatter(X_test, y_test, color='blue', label='Actual Data')
plt.plot(x_range, poly_model_original(x_range.flatten()), color='yellow', label='Polynomial Fit (Original)', linewidth=3)
plt.xlabel('X')
plt.ylabel('y')
plt.legend()
plt.title('Polynomial Fit on Test Data (Original)')

plt.show()

Il risultato evidenzia come, al costo di una minima perdita di precisione, l'idea di ridurre attraverso k-means il quantitativo di dati su cui applicare regressione sia un'idea molto efficiente.

# Importiamo il secondo dataset

In [ ]:
from sklearn.datasets import fetch_california_housing

Il dataset "California Housing" di scikit-learn contiene dati relativi ai prezzi delle case in California basati sul censimento del 1990. Questo dataset è spesso utilizzato per problemi di regressione, dove l'obiettivo è prevedere il valore medio delle case in diverse aree della California in base a una serie di caratteristiche.

In [ ]:
data=fetch_california_housing()
X,y = data.data, data.target

In [ ]:
print(X.shape, y.shape)
print(X)
print(y)
print(data.feature_names)

Il dataset contiene 20640 samples(case), ognuna con le seguenti caratteristiche (features):

1 MedInc: Reddito medio nell'area (in decine di migliaia di dollari).

2 HouseAge: Età media delle case in quell'area.

3 AveRooms: Numero medio di stanze per abitazione.

4 AveBedrms: Numero medio di camere da letto per abitazione.

5 Population: Popolazione dell'area.

6 AveOccup: Numero medio di occupanti per abitazione.

7 Latitude: Latitudine dell'area.

8 Longitude: Longitudine dell'area.

La variabile target (y) è il MedHouseVal: valore medio delle case nell'area (in centinaia di migliaia di dollari).

Iniziamo facendo un'analisi esplorativa del dataset, per cerccare di capire la distribuzione e la correlazione tra le caratteristiche del dataset in questione.

In [ ]:
# Carica il dataset California Housing
california_housing = fetch_california_housing()
df = pd.DataFrame(california_housing.data, columns=california_housing.feature_names)
df['MedHouseVal'] = california_housing.target

# Visualizza le prime righe del dataset
print(df.head())

# Descrizione statistica delle caratteristiche numeriche
print(df.describe())


# Pairplot per visualizzare le relazioni tra le caratteristiche
sns.pairplot(df, diag_kind='kde')
plt.suptitle('Pairplot delle caratteristiche del dataset California Housing', y=1.02)
plt.show()

# Heatmap delle correlazioni
plt.figure(figsize=(12, 8))
corr_matrix = df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Heatmap delle correlazioni tra le caratteristiche')
plt.show()

# Scatterplot di alcune caratteristiche rispetto al valore mediano delle case
fig, axs = plt.subplots(2, 2, figsize=(14, 12))

sns.scatterplot(data=df, x='MedInc', y='MedHouseVal', ax=axs[0, 0])
axs[0, 0].set_title('MedInc vs MedHouseVal')

sns.scatterplot(data=df, x='AveRooms', y='MedHouseVal', ax=axs[0, 1])
axs[0, 1].set_title('AveRooms vs MedHouseVal')

sns.scatterplot(data=df, x='AveOccup', y='MedHouseVal', ax=axs[1, 0])
axs[1, 0].set_title('AveOccup vs MedHouseVal')

sns.scatterplot(data=df, x='AveBedrms', y='MedHouseVal', ax=axs[1, 1])
axs[1, 1].set_title('AveBedrms vs MedHouseVal')

plt.tight_layout()
plt.show()

Dai grafici appena plottati si evidenzia che "Medinc", ossia il valore medio di reddito, è la caratteristica maggiormente correlata con il target. Inoltre si evidenzia che la caratteristica "Aveoccup" sembra essere poco rilevante al fine di predire il valore di una casa, dunque può essere rimossa al fine di ridurre la dimensionalità del dataset.
Infine le caratteristiche "AveBedrms" e "AveRooms" risultano essere altamente correlate e dunque se ne può eliminare una delle due per ridurre la multicolinearità.

In [ ]:
#rimuoviamo le due colonne dalla matrice del dataset
colonne_da_eliminare=[3,5]
X= np.delete(X,colonne_da_eliminare,axis=1)
print(X.shape)

Dopo aver fatto feature selection, cerchiamo di ridurre ulteriormente la dimensionalità del dataset(che risulta ancora alta dato che ogni sample è 6-dimensionale.

Proviamo adesso ad applicare PCA e prendiamo le prime tre componenti principali,dopo aver però verificato che esse contengono la maggior parte dell'informazione del dataset.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
#standardizziamo le caratteristiche
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Applica PCA per ridurre a 3 dimensioni
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# Varianza spiegata dalle prime 3 componenti principali
explained_variance_ratio = pca.explained_variance_ratio_
total_explained_variance = np.sum(explained_variance_ratio)

print(f'Varianza spiegata da ciascuna delle prime 3 componenti principali: {explained_variance_ratio}')
print(f'Varianza totale spiegata dalle prime 3 componenti principali: {total_explained_variance}')

# Visualizzazione della varianza spiegata
plt.figure(figsize=(8, 5))
plt.bar(range(1, 4), explained_variance_ratio, alpha=0.5, align='center', label='Singola varianza')
plt.step(range(1, 4), np.cumsum(explained_variance_ratio), where='mid', label='Varianza cumulativa')
plt.xlabel('Componenti principali')
plt.ylabel('Varianza')
plt.title('Varianza dalle prime 3 componenti principali')
plt.legend(loc='best')
plt.show()

Da tali risultati si evince che le prime tre componenti principali catturano la maggior parte dell'informazione del dataset. Risulta dunque utile e funzionale in questo caso applicare PCA.

In [ ]:
# Splittiamo il dataset in training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# K-Means

Iniziamo adesso ad addentrarci nell'idea del progetto, applicando l'algoritmo K-means, che divide il dataset in cluster e per ogni cluster identifica un "**centroide**".

Saranno proprio questi centroidi i nuovi punti su cui addestreremo un modello di regressione polinomiale.

In [ ]:
# Combiniamo X_train e y_train in un solo dataset
train_combined = np.c_[X_train, y_train]

In [ ]:
# Applichiamo K-Means
k = 2500
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(train_combined)

# Sostituiamo il training set con i centroidi
centroids = kmeans.cluster_centers_
labels = kmeans.labels_

# Splittiamo i centroidi in features e targets
X_train_reduced = centroids[:, :-1]
y_train_reduced = centroids[:, -1]

# Visualizza i primi 5 campioni con i cluster assegnati
for i in range(5):
    print(f"Campione {i+1}:")
    print(f"  Caratteristiche: {X[i]}")
    print(f"  Valore Mediano della Casa: {y[i]}")
    print(f"  Cluster Assegnato: {labels[i]}\n")

# Calcola e stampa il punteggio silhouette
silhouette_avg = silhouette_score(train_combined, kmeans.labels_)
print(f'Punteggio silhouette per k={k}: {silhouette_avg}')

# Regressione polinomiale



In [ ]:
# Fittiamo un modello sul training set formato dai centroidi
degree = 60
X_train_reduced_1d = X_train_reduced[:, 0]
coefs_reduced = np.polyfit(X_train_reduced_1d, y_train_reduced, degree)
polyfit_model_reduced = np.poly1d(coefs_reduced)

# Fittiamo un modello sul training set originale
X_train_1d = X_train[:, 0]
coefs_original = np.polyfit(X_train_1d, y_train, degree)
polyfit_model_original = np.poly1d(coefs_original)

# valutiamo sul test set entrambi i modelli
X_test_1d = X_test[:, 0]
y_pred_reduced = polyfit_model_reduced(X_test_1d)
y_pred_original = polyfit_model_original(X_test_1d)

# Calculiamo l'MSE
mse_reduced = mean_squared_error(y_test, y_pred_reduced)
mse_original = mean_squared_error(y_test, y_pred_original)

# Calculiamo R²
r2_reduced = r2_score(y_test, y_pred_reduced)
r2_original = r2_score(y_test, y_pred_original)

print("Mean Squared Error on Test Set (Reduced):", mse_reduced)
print("R² Score on Test Set (Reduced):", r2_reduced)

print("Mean Squared Error on Test Set (Original):", mse_original)
print("R² Score on Test Set (Original):", r2_original)

Non sembra un buon risultato :( , proviamo dunque a ripetere lo stesso procedimento ma senza utilizzare PCA.

In [ ]:
# Splittiamo il dataset in training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# K-Means

In [ ]:
# Combiniamo X_train e y_train in un solo dataset
train_combined = np.c_[X_train, y_train]

In [ ]:
# Applichiamo K-means
k = 2500
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(train_combined)

# Sostituiamo il training set con i centroidi
centroids = kmeans.cluster_centers_
labels = kmeans.labels_

# Splittiamo i centroidi in features e targets
X_train_reduced = centroids[:, :-1]
y_train_reduced = centroids[:, -1]

# Visualizza i primi 5 campioni con i cluster assegnati
for i in range(5):
    print(f"Campione {i+1}:")
    print(f"  Caratteristiche: {X[i]}")
    print(f"  Valore Mediano della Casa: {y[i]}")
    print(f"  Cluster Assegnato: {labels[i]}\n")

# Calcola e stampa il punteggio silhouette
silhouette_avg = silhouette_score(train_combined, kmeans.labels_)
print(f'Punteggio silhouette per k={k}: {silhouette_avg}')

# Regressione polinomiale



In [ ]:
# Fittiamo un modello sul training set formato dai centroidi
degree = 60
X_train_reduced_1d = X_train_reduced[:, 0]
coefs_reduced = np.polyfit(X_train_reduced_1d, y_train_reduced, degree)
polyfit_model_reduced = np.poly1d(coefs_reduced)

# Fittiamo un modello sul training set oroginale
X_train_1d = X_train[:, 0]
coefs_original = np.polyfit(X_train_1d, y_train, degree)
polyfit_model_original = np.poly1d(coefs_original)

# Valutiamo entrambi i modelli sul test set
X_test_1d = X_test[:, 0]
y_pred_reduced = polyfit_model_reduced(X_test_1d)
y_pred_original = polyfit_model_original(X_test_1d)

# Calcoliamo l'MSE
mse_reduced = mean_squared_error(y_test, y_pred_reduced)
mse_original = mean_squared_error(y_test, y_pred_original)

# Calcoliamo R²
r2_reduced = r2_score(y_test, y_pred_reduced)
r2_original = r2_score(y_test, y_pred_original)

print("Mean Squared Error on Test Set (Reduced):", mse_reduced)
print("R² Score on Test Set (Reduced):", r2_reduced)

print("Mean Squared Error on Test Set (Original):", mse_original)
print("R² Score on Test Set (Original):", r2_original)

Ecco che qui abbiamo un risultato molto più soddisfacente: infatti l'MSE risulta essere molto basso e l'R^2 ha un valore buono(vicino a 0.5).

Inoltre possiamo vedere come si abbia una differenza minima tra i risultati ottenuti con la classica regressione polinomiale e quella fatta con i centroidi.

# Importiamo il terzo dataset

Il dataset "**Epidemy diffusion**" è un dataset costruito da noi che descrive l'evolversi di un'epidemia di una popolazione lungo un certo periodo di tempo(in anni).

La spiegazione dettagliata del dataset(con relativo codice) è presente su un file che manderemo in allegato per email.

In [ ]:
# Load the Data
data = np.loadtxt('epidemy_diffusion_3.0.dat')

X = data[:, :-1]
y = data[:, -1]
print(X.shape, y.shape)

In [ ]:
# Splittiamo il dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# K-Means

In [ ]:
# Applichiamo K-Means
k = 80
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X_train)

centroids = kmeans.cluster_centers_

labels = kmeans.predict(X_train)

# prendiamo i valori target dei centroidi facendo la media dei valori target in ogni cluster
y_centroids = np.array([y_train[labels == i].mean() for i in range(k)])


# Sostituiamo il training Set con i centroidi
X_train_centroids = centroids
y_train_centroids = y_centroids


# Calcola e stampa il punteggio silhouette
silhouette_avg = silhouette_score(X_train, kmeans.labels_)
print(f'Punteggio silhouette per k={k}: {silhouette_avg}')

# Regressione polinomiale

In [ ]:
#  fit polinomiale sui centroidi
degree = 25
poly_model_centroids = Polynomial.fit(X_train_centroids.flatten(), y_train_centroids, degree)

# fit sul training set originale
poly_model_original = Polynomial.fit(X_train.flatten(), y_train, degree)

# valutiamo i modelli
y_pred_test_centroids = poly_model_centroids(X_test.flatten())
y_pred_test_original = poly_model_original(X_test.flatten())

# R^2
r2_test_centroids = r2_score(y_test, y_pred_test_centroids)
r2_test_original = r2_score(y_test, y_pred_test_original)

# MSE
mse_test_centroids = mean_squared_error(y_test, y_pred_test_centroids)
mse_test_original = mean_squared_error(y_test, y_pred_test_original)

# Print dei risultati
print(f'\nR^2 on test data (centroids): {r2_test_centroids}')
print(f'R^2 on test data (original): {r2_test_original}')
print(f'MSE on test data (centroids): {mse_test_centroids}')
print(f'MSE on test data (original): {mse_test_original}')

# Plottiamo il training set e i centroidi
plt.figure(figsize=(14, 7))

plt.subplot(2, 2, 1)
plt.scatter(X_train, y_train, color='green', label='Original Training Set', s=5)
plt.xlabel('X')
plt.ylabel('y')
plt.legend()
plt.title('Original Training Set')

plt.subplot(2, 2, 2)
plt.scatter(X_train_centroids, y_train_centroids, color='red', label='Centroids Set', s=10)
plt.xlabel('X')
plt.ylabel('y')
plt.legend()
plt.title('Centroids Set')

plt.figure(figsize=(14, 7))

x_range = np.linspace(X_test.min(), X_test.max(), 500)

plt.subplot(1, 2, 1)
plt.scatter(X_test, y_test, color='blue', label='Actual Data', s=20)
plt.plot(x_range, poly_model_centroids(x_range.flatten()), color='red', linewidth=2.5, label='Polynomial Fit (Centroids)')
plt.xlabel('X')
plt.ylabel('y')
plt.legend()
plt.title('Polynomial Fit on Test Data (Centroids)')

plt.subplot(1, 2, 2)
plt.scatter(X_test, y_test, color='blue', label='Actual Data', s=20)
plt.plot(x_range, poly_model_original(x_range.flatten()), color='green', linewidth=2.5, label='Polynomial Fit (Original)')
plt.xlabel('X')
plt.ylabel('y')
plt.legend()
plt.title('Polynomial Fit on Test Data (Original)')

plt.show()

In questo caso ,utilizzando il fit con i centroidi rispetto al classico, si nota addirittura un piccolissimo guadagno in termini di precisione, come evidenziato dai valori MSE e R^2

# Importiamo il quarto dataset

In [ ]:
from sklearn.datasets import load_linnerud

Il dataset "**Linnerud**" di scikit-learn è un dataset multidimensionale che contiene misurazioni relative all'esercizio fisico e ai parametri fisiologici.

Caratteristiche (Features):

Il dataset include 3 caratteristiche che misurano l'attività fisica dei soggetti:

1. Chins: Numero di flessioni alla sbarra (pull-ups).

2. Sit-ups: Numero di addominali eseguiti in 60 secondi.

3. Jumps: Distanza massima di salto in centimetri.

Queste caratteristiche sono indicazioni delle capacità fisiche e della forma fisica generale dei soggetti.

Target:

Il dataset include 3 variabili target che rappresentano parametri fisiologici dei soggetti:

1. Weight: Peso del soggetto in kg.

2. Waist: Circonferenza della vita in cm.

3. Pulse: Frequenza cardiaca a riposo in battiti per minuto (bpm).

Queste variabili target sono misure fisiologiche che possono essere influenzate dal livello di attività fisica del soggetto.

In [ ]:
data = load_linnerud()
X = data.data
y = data.target[:, 0]  # Consideriamo solo il peso come target per semplicità
print(X.shape,y.shape)

Adesso cerchiamo di ridurre la dimensionalità del dataset a 2 dimensioni, attraverso PCA e previa analisi della varianza contenuta  nelle prime due componenti principali, in modo da poter poi visualizzare in 3D il fit polinomiale(per questo abbiamo il target ad una sola componente)

In [ ]:
# Applicare PCA per ridurre le dimensioni a 2
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

# Verificare quanta varianza contengono le prime due componenti principali
explained_variance = pca.explained_variance_ratio_
print(f'Varianza spiegata dalle prime due componenti principali: {explained_variance}')# Dividere il dataset in training e test set
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

Come si vede dunque le prime due componenti principali catturano la quasi totalità dell'informazione del dataset.

In [ ]:
# Dividiamo il dataset in training e test set
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

# K-Means

In [ ]:
train_combined = np.c_[X_train, y_train]

In [ ]:
k = 3
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(train_combined)

centroids = kmeans.cluster_centers_

X_train_reduced = centroids[:, :-1]
y_train_reduced = centroids[:, -1]

silhouette_avg = silhouette_score(train_combined, kmeans.labels_)
print(f'Punteggio silhouette per k={k}: {silhouette_avg}')

# Regressione polinomiale

In [ ]:
# Trasformiamo le caratteristiche in caratteristiche polinomiali
degree = 2  # Grado del polinomio
poly = PolynomialFeatures(degree=degree)
X_train_poly = poly.fit_transform(X_train_reduced)
X_test_poly = poly.transform(X_test)

# Applicare la regressione lineare sulle caratteristiche polinomiali
model = LinearRegression()
model.fit(X_train_poly, y_train_reduced)

# Predire i valori
y_train_pred = model.predict(X_train_poly)
y_test_pred = model.predict(X_test_poly)

# Calcolare l'MSE e l'R^2 per i dati di training
train_mse = mean_squared_error(y_train_reduced, y_train_pred)
train_r2 = r2_score(y_train_reduced, y_train_pred)

# Calcolare l'MSE e l'R^2 per i dati di test
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Train MSE: {train_mse}')
print(f'Train R^2: {train_r2}')
print(f'Test MSE: {test_mse}')
print(f'Test R^2: {test_r2}')

# Visualizzare il fit polinomiale sui punti in un grafico 3D interattivo con Plotly
# Punti originali
scatter = go.Scatter3d(
    x=X_reduced[:, 0],
    y=X_reduced[:, 1],
    z=y,
    mode='markers',
    marker=dict(size=5, color='blue', opacity=0.8),
    name='Dati originali'
)

# Creare una griglia di punti per visualizzare il fit polinomiale
x0_range = np.linspace(X_reduced[:, 0].min(), X_reduced[:, 0].max(), 100)
x1_range = np.linspace(X_reduced[:, 1].min(), X_reduced[:, 1].max(), 100)
x0_grid, x1_grid = np.meshgrid(x0_range, x1_range)
x_grid = np.c_[x0_grid.ravel(), x1_grid.ravel()]
x_grid_poly = poly.transform(x_grid)
y_grid_pred = model.predict(x_grid_poly).reshape(x0_grid.shape)

# Superficie del fit polinomiale
surface = go.Surface(
    x=x0_grid,
    y=x1_grid,
    z=y_grid_pred,
    colorscale='Viridis',
    opacity=0.5,
    name='Superficie del fit polinomiale'
)

layout = go.Layout(
    title='Fit Polinomiale sui Dati Linnerud',
    scene=dict(
        xaxis_title='Prima componente principale',
        yaxis_title='Seconda componente principale',
        zaxis_title='Peso'
    )
)

fig = go.Figure(data=[scatter, surface], layout=layout)
fig.show()

Come si evince da questi risultati il modello fitta benissimo i dati di training dato che il grado del polinomio è >= (n_cluster - 1), ma ha performance molto scadenti sul test set. Si ha dunque un overfitting, a causa della ridotta quantità di punti presenti nel dataset.

Ecco dunque un esempio in cui l'idea di utilizzare k-means non produce buoni risultati. Fortunamente però il campo di applicazione previsto per questa procedura è costituito da dataset estremamente grandi, come si è potuto constatare in precedenza.

# Generiamo l'ultimo dataset: una curva

Come ultimo dataset utilizzeremo dei punti generati dalla seguente curva,detta curva di Sine, definita come:
              
$$f(t) = (t, \sin(t), \sin(2t))\,,\quad\quad t \in [0, 5\pi]$$




In [ ]:
t = np.linspace(0, 2 * np.pi, 100)
x = t
y = np.sin(t)
z = np.sin(2 * t)

# Plot della curva in R^3
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

ax.plot(x, y, z, label='(t, sin(t), sin(2*t)', color='red', linewidth=4)

ax.set_xlabel('t')
ax.set_ylabel('sin(t)')
ax.set_zlabel('sin(2*t)')

ax.set_title('Curva (t, sin(t), sin(2*t)) in R^3')
ax.legend()

plt.show()

In reatà però quello che faremo non sarà prendere dei normali punti della curva,ma sarà prendere dei punti dalla curva perturbata.

In [ ]:
# Generazione del dataset con perturbazione
np.random.seed(0)
t = np.linspace(0, 2 * np.pi, 100)
x = t
y = np.sin(t) + 0.1 * np.random.randn(100)
z = np.sin(2 * t) + 0.1 * np.random.randn(100)

# Dataset
data = np.vstack((x, y, z)).T

Adesso splittiamo il dataset in training set e test set

In [ ]:
# Divisione del dataset in training e test set
train_data, test_data = train_test_split(data, test_size=0.3, random_state=0)

# K-Means

In [ ]:
# applichiamo k-means
kmeans = KMeans(n_clusters=10, random_state=0).fit(train_data)
train_labels = kmeans.labels_
test_labels = kmeans.predict(test_data)

# prendiamo i centroidi
centroids = kmeans.cluster_centers_


Adesso applicheremo regressione polinomiale sul dataset originale e su quello formato dai centroidi, cercando di cogliere le differenze tra i due modelli

# Regressione polinomiale

In [ ]:
# Regressione polinomiale sui centroidi
poly = PolynomialFeatures(degree=5)
centroids_poly = poly.fit_transform(centroids[:, :2])

model_centroids = LinearRegression()
model_centroids.fit(centroids_poly, centroids[:, 2])

# Predizione sui dati di test utilizzando il modello basato sui centroidi
X_test_poly = poly.transform(test_data[:, :2])
z_test_pred_centroids = model_centroids.predict(X_test_poly)

# Regressione polinomiale diretta
X_train_poly = poly.fit_transform(train_data[:, :2])
model = LinearRegression()
model.fit(X_train_poly, train_data[:, 2])
z_test_pred = model.predict(X_test_poly)

# MSE e R^2 per regressione polinomiale diretta e con centroidi
mse_direct = mean_squared_error(test_data[:, 2], z_test_pred)
r2_direct = r2_score(test_data[:, 2], z_test_pred)
mse_centroids = mean_squared_error(test_data[:, 2], z_test_pred_centroids)
r2_centroids = r2_score(test_data[:, 2], z_test_pred_centroids)

Stampiamo adesso MSE e R^2 e facciamo un plot del fit di entrambi i modelli

In [ ]:
# Plotting
fig = plt.figure(figsize=(12, 6))

# Plot per regressione polinomiale diretta
ax = fig.add_subplot(121, projection='3d')
ax.scatter(test_data[:, 0], test_data[:, 1], test_data[:, 2], c='r', marker='o', label='Dati Test Originali')

# Generazione di una griglia per il fit polinomiale
x_range = np.linspace(test_data[:, 0].min(), test_data[:, 0].max(), 100)
y_range = np.linspace(test_data[:, 1].min(), test_data[:, 1].max(), 100)
x_grid, y_grid = np.meshgrid(x_range, y_range)
xy_grid = np.c_[x_grid.ravel(), y_grid.ravel()]
z_grid = model.predict(poly.transform(xy_grid)).reshape(x_grid.shape)

ax.plot_trisurf(xy_grid[:, 0], xy_grid[:, 1], model.predict(poly.transform(xy_grid)), alpha=0.5, color='b')
ax.set_title('Regressione Polinomiale Diretta')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

# Plot per regressione polinomiale con centroidi
ax = fig.add_subplot(122, projection='3d')
ax.scatter(test_data[:, 0], test_data[:, 1], test_data[:, 2], c='r', marker='o', label='Dati Test Originali')

# Generazione di una griglia per il fit polinomiale con centroidi
ax.plot_trisurf(xy_grid[:, 0], xy_grid[:, 1], model_centroids.predict(poly.transform(xy_grid)), alpha=0.5, color='g')
ax.set_title('Regressione Polinomiale con Centroidi')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

plt.show()

# Stampa di MSE e R^2
print(f'MSE Regressione Polinomiale Diretta (Test): {mse_direct}')
print(f'R^2 Regressione Polinomiale Diretta (Test): {r2_direct}')
print(f'MSE Regressione Polinomiale con Centroidi (Test): {mse_centroids}')
print(f'R^2 Regressione Polinomiale con Centroidi (Test): {r2_centroids}')

Anche in questo caso al costo di una piccola perdita di precisione si ottengono degli ottimi risultati, come evidenziato dal plot della superficie e dai valori di MSE e R^2.

# Conclusioni finali

Dopo aver esaminato alcuni dataset siamo giunti alla conclusione che il metodo descritto nel progetto ha un buon potenziale di applicazione, in particolare nel caso di dataset abbastanza grandi e adatti alla clusterizzazione.

Speriamo con ENORME umiltà che questa(a dir poco) rivoluzionaria idea possa portare a grandi innovazioni nel campo del machine learning😂

Fonti: Chatgpt, scikit-learn, wikipedia, lezioni del docente E. Rodolà.